In [ ]:
!pip i

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import faiss



In [2]:
# Set the paths
dataset_version = "lite"  # choose "lite" or "full"
unsplash_dataset_path = Path("unsplash-dataset") / dataset_version
features_path = Path("unsplash-dataset") / dataset_version / "features"

# Read the photos table
photos = pd.read_csv(unsplash_dataset_path / "photos.tsv000", sep='\t', header=0)

# Load the features and the corresponding IDs
photo_features = np.load(features_path / "features.npy")
photo_ids = pd.read_csv(features_path / "photo_ids.csv")
photo_ids = list(photo_ids['photo_id'])


# 38 photos have same embeddings
photo_features = np.unique(photo_features, axis=0)


In [3]:
photo_features

array([[-0.08344765,  0.01604631,  0.03037113, ...,  0.03962869,
        -0.02023221, -0.01102292],
       [-0.08083483,  0.02393946,  0.00666234, ...,  0.02837397,
        -0.0522343 , -0.0081386 ],
       [-0.07816768,  0.0393783 , -0.01087704, ...,  0.04282344,
        -0.02091642, -0.01027706],
       ...,
       [ 0.08788611,  0.03488644, -0.01199933, ...,  0.01401449,
         0.00184289, -0.01959554],
       [ 0.08824386,  0.01944975,  0.01758924, ...,  0.03036399,
        -0.00658768, -0.03173414],
       [ 0.09195293,  0.04004934, -0.00255247, ...,  0.00362209,
        -0.01816886, -0.04212745]], dtype=float32)

In [4]:
#new_data = np.unique(photo_features, axis=0)


In [5]:
#len(new_data) , len(photo_features)

In [6]:
photo_ids[:3]

['br4dnLzZlXg', 'YTB3TFbz19w', 'g-Gc-k-NPkA']

In [7]:
k = 3

In [8]:
xb = photo_features.tolist()
xb = np.array(xb).astype('float32')

In [9]:
xb.shape


(24957, 512)

In [10]:
d = xb.shape[-1]                       
d

512

In [11]:
index = faiss.IndexFlatL2(d)


In [12]:
index.is_trained


True

In [13]:
index.add(xb)                


In [14]:
xq = xb


In [15]:
len(xq)


24957

In [16]:
%%time
D, I = index.search(xq[:1], k)   

CPU times: user 20.7 ms, sys: 0 ns, total: 20.7 ms
Wall time: 19 ms


In [17]:
%%time
D, I = index.search(xq, k)     

CPU times: user 32 s, sys: 35.9 ms, total: 32 s
Wall time: 10.7 s


In [18]:
D


array([[0.00000000e+00, 2.62009501e-01, 2.68038988e-01],
       [0.00000000e+00, 1.42064452e-01, 1.52374268e-01],
       [1.19209290e-07, 1.63132429e-01, 1.83581352e-01],
       ...,
       [0.00000000e+00, 8.58912468e-02, 1.00482225e-01],
       [8.34465027e-07, 2.90696144e-01, 3.25628757e-01],
       [2.38418579e-07, 4.94306803e-01, 5.24386048e-01]], dtype=float32)

In [19]:
D

array([[0.00000000e+00, 2.62009501e-01, 2.68038988e-01],
       [0.00000000e+00, 1.42064452e-01, 1.52374268e-01],
       [1.19209290e-07, 1.63132429e-01, 1.83581352e-01],
       ...,
       [0.00000000e+00, 8.58912468e-02, 1.00482225e-01],
       [8.34465027e-07, 2.90696144e-01, 3.25628757e-01],
       [2.38418579e-07, 4.94306803e-01, 5.24386048e-01]], dtype=float32)

In [20]:
I

array([[    0,   416,  1119],
       [    1,   686,   133],
       [    2,    22,   220],
       ...,
       [24954, 24896, 24874],
       [24955, 15230, 13425],
       [24956, 22310, 23279]])

In [21]:
z = I[:,0] == np.array( list(range(len(xq))))

In [22]:
z.sum() , len(z) ,  (  len(z) - z.sum())

(24957, 24957, 0)

In [ ]:
np.array( list(range(len(xq))))[z]

In [ ]:
z

In [ ]:
np.where(z == False)

In [ ]:
D[18]

In [ ]:
I[18]

In [41]:
24957 / nlist

249.57

## faiss ivf

In [42]:
nlist = 20 # nmber of clusters

quantizer = faiss.IndexFlatL2(d)  # the other index
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)

In [43]:
assert not index.is_trained
index.train(xb)
assert index.is_trained

In [44]:
xq = xb
index.add(xb)         


In [45]:
%%time


index.nprobe = 1              # default nprobe is 1

D, I = index.search(xq[:1], k)     # actual search

CPU times: user 0 ns, sys: 1.57 ms, total: 1.57 ms
Wall time: 661 µs


In [46]:
%%time


index.nprobe = 1              # default nprobe is 1

D, I = index.search(xq, k)     # actual search

CPU times: user 28.3 s, sys: 548 µs, total: 28.3 s
Wall time: 3.63 s


In [47]:
z = I[:,0] == np.array( list(range(len(xq))))
z.sum() , len(z) ,  (  len(z) - z.sum())

(24957, 24957, 0)